### 1) Tips from Jule

In [1]:
#libraries
import pandas as pd

In [1]:
# loading csv into dataframe

def load_data(path):
    return pd.read_csv(path)

e.g. 
df2 = pd.read_csv("customer_analysis_clean.csv")
df2

In [ ]:
# Transform data

def transform_data(df):
    df["total_claim_amount"]=round(df["total_claim_amount"],0) # remove decimals
    df["state"]=df["state"].apply(state_region) # change the column name state into the column name state_region
    
    return df

# dont forget to ultimately apply the function on the relevant df such as df = transform_data(df)

In [2]:
# cleaning

def clean_data(df):
    df.clouns = [e.lower().replace(" ", "_") for e in df.columns] #all columns in lower case
    df=df.drop(columns=["unnamed:_0", "vehicle_type", "customer"]) #dropping useless columns
    df["effective_to_date"]=pd.to_datetime(df["effective_to_date"], errors="coerce") # change formate to date
    df["month"]=list(map(lambda date:date.strftime(format="%B"),df["effective_to_date"])) #get from effective to date just the months (%A would be day)
    df=df.drop(columns=["effective_to_date"]) #again dropping unnecessary columns
    df["state"] = df["state"].apply(st_clean)
    df["gender"] = df["gender"].apply(gender_clean) # both probably changing the name of the column from the former to the new
    df = df.drop_duplicates() #dropping all the entirely same rows in the df and overwrite the old version with the new one
    df = df.reset_index(drop=True) #ensuring that the amount of index is reduced to the new/correct amount after dropping some rows
    df = df.dropna() #drop if any cell is empty
    df["customer_lifetime_value"].fillna(df["customer_lifetime_value"].mean(),inplace=True) #fill empty cells of a column with its respective average
    df["vehicle_type"] = df["vehicle_type"].fillna(value="M") #fill empty cells of a column with the M value only
    df["number_of_open_complaints"] = df["number_of_open_complaints"].fillna(value=0) #fill empty cells of a column with 0 value only
    
    return df

# dont forget to ultimately apply the function on the relevant df such as df = clean_data(df)
# below we have a BETTER SPLIT SOLUTION:

### 2) OWN WAY DONE
See own Pipeline at the beginning of -> http://localhost:8888/notebooks/Documents/GitHub/IronAlfred/LAB_Vol_2_Linear_Regression_M_Day18/LAB_Vol_2_Linear_Regression_M_Day18.ipynb

In [ ]:
#libraries
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
%matplotlib inline
from numpy import random
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sc
from ipywidgets import interactive
import ipywidgets as widgets
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import boxcox
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#Function for reading the data:
def read_customer_data():
    return pd.read_csv("Data_Marketing_Customer_Analysis_Round2.csv")

In [ ]:
df = read_customer_data()

In [ ]:
#Function for standardizing column names:
def standardize(df):
    new_columns = df.columns.str.lower().str.replace(" ", "_")
    df.columns = new_columns
    return df

In [ ]:
#Function for dropping customer and unnamed:_0 columns
def drop_2cols(df):
    df.drop(columns=["unnamed:_0", "customer"], inplace=True)
    return df

In [ ]:
#Drop duplicates and reset index
def dd(df):
    df = df.drop_duplicates().reset_index()
    df.drop(columns=["index"], inplace=True)
    return df

In [ ]:
#Replacing null values
def fill_null(df):
    df["number_of_open_complaints"] = df["number_of_open_complaints"].fillna(value=0)
    df["vehicle_type"] = df["vehicle_type"].fillna(value="M")
    mean_months = round(df["months_since_last_claim"].mean())
    df["months_since_last_claim"] = df["months_since_last_claim"].fillna(value=mean_months)
    mean_income = round(df["income"].mean())
    df["income"] = df["income"].fillna(value=mean_income)
    return df

In [ ]:
#Creating a months column for the effective_to_date column only
def month_creation(df):
    df['effective_to_date']=pd.to_datetime(df['effective_to_date'], errors='coerce') #turn from object to datetime format
    df["month_effective_to_date"]=list(map(lambda date:date.strftime(format="%B"),df["effective_to_date"]))
    return(df)

In [ ]:
# RUNNING THE (rest) OF THE PIPELINE (and at the end have 2 closer looks at it)

In [ ]:
df = standardize(df)

In [ ]:
df = month_creation(df)

In [ ]:
df= drop_2cols(df)

In [ ]:
df = dd(df)

In [ ]:
df = fill_null(df)

In [ ]:
df.info()

In [ ]:
df

### 3) NEW STUFF FROM MIDTERM
-> http://localhost:8888/notebooks/Documents/GitHub/Iron_Midterm/Olist%20-%20Data%20Cleaning.ipynb

In [ ]:
# Activate the DB connection from SQL into Jupyter Notebook

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'

connection_string = f'{driver}://{user}:{password}@{ip}'
    
db_connection = sa.create_engine(connection_string)

insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()


In [ ]:
# Pull closed_deals column data from Schema Olist into Jupyter Notebook

query = "SELECT * FROM olist.closed_deals"
closed_deals = pd.read_sql_query(query, db_connection)

In [ ]:
# change/reduce datetime: removal of time in the timestemp to speed up calculations (date remains)
closed_deals["won_date"] = pd.to_datetime(closed_deals["won_date"]).dt.date

In [ ]:
#to push back to workbench:
closed_deals.to_sql(con=db_connection, schema='olist', name='closed_deals', if_exists='replace', index = False)

In [ ]:
#seller_city one has a number in the cell and should actually be rio de janeiro: 04482255 -> Replace value with an other
sellers = sellers.replace(to_replace='04482255', value = 'rio de janeiro')

In [ ]:
# add a new column called seller_state_name into the existing table on column position Nr. 4 with the same values as an other column (for now)
seller_state_name = sellers.seller_state

sellers.insert(loc=4,
          column='seller_state_name',
          value=seller_state_name)
sellers.head(2)
# function on the bottom shows the 2 first records of the new table

In [ ]:
# Fill the new column with new values -> find and replace

def full_name(x):
    if not x == x:
        return x
    else:
        name = {'RO': 'Rondônia', 
'AC': 'Acre', 
'AM': 'Amazonas', 
'RR': 'Roraima', 
'PA': 'Pará', 
'AP': 'Amapá', 
'TO': 'Tocantins', 
'MA': 'Maranhão', 
'PI': 'Piauí', 
'CE': 'Ceará', 
'RN': 'Rio Grande do Norte', 
'PB': 'Paraíba', 
'PE': 'Pernambuco', 
'AL': 'Alagoas', 
'SE': 'Sergipe', 
'BA': 'Bahia', 
'MG': 'Minas Gerais', 
'ES': 'Espírito Santo', 
'RJ': 'Rio de Janeiro', 
'SP': 'Sao Paulo', 
'PR': 'Paraná', 
'SC': 'Santa Catarina', 
'RS': 'Rio Grande do Sul', 
'MS': 'Mato Grosso do Sul', 
'MT': 'Mato Grosso', 
'GO': 'Goiás', 
'DF': 'Distrito Federal'}
        return name[x]

In [ ]:
# execute the renaming function abouve to the new column called seller_state_name
sellers.seller_state_name = sellers.seller_state_name.apply(full_name)

In [ ]:
#bucketing "processing & approved, invoiced"  as these orders have not been processed correctly: new name "unprocessed"; rename "shipped" "lost_during_delivery"

orders.order_status[orders.order_status.isin(["processing","invoiced", "approved"])] = "unprocessed"
orders.order_status[orders.order_status.isin(["shipped"])] = "lost_during_delivery"
orders.order_status.unique()
# function on the bottom showsthe newly unique values in the order_status column

In [ ]:
#change product_catgeory_name to english, whereas the was a lookup table, which we had to bring into a Dictionary format via Google Sheet

def change_name(x):
    if not x == x:
        return x
    else:
        category = {"beleza_saude": "health_beauty",
        "informatica_acessorios":"computers_accessories",
        "automotivo":"auto",
        "cama_mesa_banho":"bed_bath_table",
        "moveis_decoracao":"furniture_decor",
        "esporte_lazer":"sports_leisure",
        "perfumaria":"perfumery",
        "utilidades_domesticas":"housewares",
        "telefonia":"telephony",
        "relogios_presentes":"watches_gifts",
        "alimentos_bebidas":"food_drink",
        "bebes":"baby",
        "papelaria":"stationery",
        "tablets_impressao_imagem":"tablets_printing_image",
        "brinquedos":"toys",
        "telefonia_fixa":"fixed_telephony",
        "ferramentas_jardim":"garden_tools",
        "fashion_bolsas_e_acessorios":"fashion_bags_accessories",
        "eletroportateis":"small_appliances",
        "consoles_games":"consoles_games",
        "audio":"audio",
        "fashion_calcados":"fashion_shoes",
        "cool_stuff":"cool_stuff",
        "malas_acessorios":"luggage_accessories",
        "climatizacao":"air_conditioning",
        "construcao_ferramentas_construcao":"construction_tools_construction",
        "moveis_cozinha_area_de_servico_jantar_e_jardim":"kitchen_dining_laundry_garden_furniture",
        "construcao_ferramentas_jardim":"costruction_tools_garden",
        "fashion_roupa_masculina":"fashion_male_clothing",
        "pet_shop":"pet_shop",
        "moveis_escritorio":"office_furniture",
        "market_place":"market_place",
        "eletronicos":"electronics",
        "eletrodomesticos":"home_appliances",
        "artigos_de_festas":"party_supplies",
        "casa_conforto":"home_confort",
        "construcao_ferramentas_ferramentas":"costruction_tools_tools",
        "agro_industria_e_comercio":"agro_industry_and_commerce",
        "moveis_colchao_e_estofado":"furniture_mattress_and_upholstery",
        "livros_tecnicos":"books_technical",
        "casa_construcao":"home_construction",
        "instrumentos_musicais":"musical_instruments",
        "moveis_sala":"furniture_living_room",
        "construcao_ferramentas_iluminacao":"construction_tools_lights",
        "industria_comercio_e_negocios":"industry_commerce_and_business",
        "alimentos":"food",
        "artes":"art",
        "moveis_quarto":"furniture_bedroom",
        "livros_interesse_geral":"books_general_interest",
        "construcao_ferramentas_seguranca":"construction_tools_safety",
        "fashion_underwear_e_moda_praia":"fashion_underwear_beach",
        "fashion_esporte":"fashion_sport",
        "sinalizacao_e_seguranca":"signaling_and_security",
        "pcs":"computers",
        "artigos_de_natal":"christmas_supplies",
        "fashion_roupa_feminina":"fashio_female_clothing",
        "eletrodomesticos_2":"home_appliances_2",
        "livros_importados":"books_imported",
        "bebidas":"drinks",
        "cine_foto":"cine_photo",
        "la_cuisine":"la_cuisine",
        "musica":"music",
        "casa_conforto_2":"home_comfort_2",
        "portateis_casa_forno_e_cafe":"small_appliances_home_oven_and_coffee",
        "cds_dvds_musicais":"cds_dvds_musicals",
        "dvds_blu_ray":"dvds_blu_ray",
        "flores":"flowers",
        "artes_e_artesanato":"arts_and_craftmanship",
        "fraldas_higiene":"diapers_and_hygiene",
        "fashion_roupa_infanto_juvenil":"fashion_childrens_clothes",
        "seguros_e_servicos":"security_and_services",
         "pc_gamer":"pc_gamer", 
         "portateis_cozinha_e_preparadores_de_alimentos":"kitchen_appliance"}

        return category[x]
    

In [ ]:
#bucketing of the categories: from 73 to 21 buckets, making especially sure that nothing, even if no bucketing involved is missed out

products.product_category_name[products.product_category_name.isin(['christmas_supplies','cool_stuff','party_supplies'])] = "event_supplies"
products.product_category_name[products.product_category_name.isin(['bed_bath_table','furniture_bedroom','furniture_decor','furniture_living_room', 'furniture_mattress_and_upholstery', 'kitchen_dining_laundry_garden_furniture'])] = "furniture"
products.product_category_name[products.product_category_name.isin(['art','arts_and_craftmanship'])] = "arts"  
products.product_category_name[products.product_category_name.isin(['books_general_interest','books_imported', 'books_technical'])] = "books" 
products.product_category_name[products.product_category_name.isin(['cds_dvds_musicals','dvds_blu_ray','audio','music','musical_instruments'])] = "video_audio_instruments"  
products.product_category_name[products.product_category_name.isin(['electronics', 'computers','computers_accessories','fixed_telephony','cine_photo','telephony'])] = "electronics"      
products.product_category_name[products.product_category_name.isin(['construction_tools_lights', 'construction_tools_safety', 'costruction_tools_garden','costruction_tools_tools','construction_tools_construction','home_construction'])] = "construction_tools"                                
products.product_category_name[products.product_category_name.isin(['fashio_female_clothing','fashion_bags_accessories','fashion_childrens_clothes','fashion_male_clothing','fashion_shoes','fashion_underwear_beach','luggage_accessories'])] = "fashion"   
products.product_category_name[products.product_category_name.isin(['baby','diapers_and_hygiene'])] = "baby"                                 
products.product_category_name[products.product_category_name.isin(['food','food_drink','drinks'])] = "food_drinks"                                                                 
products.product_category_name[products.product_category_name.isin(["kitchen_appliance", "small_appliances_home_oven_and_coffee", "la_cuisine"])] = "kitchen"
products.product_category_name[products.product_category_name.isin(["perfumery", "health_beauty"])] = "health_beauty"
products.product_category_name[products.product_category_name.isin(["security_and_services", "signaling_and_security"])] = "security"
products.product_category_name[products.product_category_name.isin(["flowers", "garden_tools", "agro_industry_and_commerce"])] = "garden"
products.product_category_name[products.product_category_name.isin(['pet_shop'])] = "pets"
products.product_category_name[products.product_category_name.isin(["office_furniture", "industry_commerce_and_business", "stationery", "tablets_printing_image"])] = "office"
products.product_category_name[products.product_category_name.isin(["pc_gamer", "consoles_games", "toys"])] = "games"
products.product_category_name[products.product_category_name.isin(["small_appliances", "home_appliances", "home_appliances_2", "home_comfort_2", "home_confort", "housewares", "air_conditioning"])] = "home_appliances"                               
products.product_category_name[products.product_category_name.isin(['watches_gifts'])] = "watches"  
products.product_category_name[products.product_category_name.isin(['fashion_sport','sports_leisure'])] = "sports"

In [ ]:
#### CLEANING DATA IN SQL

#### Drop not needed table (just via right click in the schema section in Workbench or via code below)

#Code:
#drop table `order_status_year_price`
#drop table `prod_cat_price`
#drop table `product_category_name_translation`
#drop table `geolocation`

## Delete rows without value 

#in payment_type: delete rows where type = 'not_defined' as payment value of these three order = 0€
#Code: delete from olist.order_payments where order_id in ('4637ca194b6387e2d538dc89b124b0ee','00b1cb0320190ca0daa2c88b35206009','c8c528189310eaa44a745b8d9d26908b')

# in order_customer_items_paid delete rows with status "shipped" --> we have analysed that these order has been lost during delivery and most likely the customer git a refund
#Code: delete from order_customer_items_paid where order_status in ('shipped')

# data from september 2018 not complete, therfore we will delete this month
#Code: delete from orders where order_purchase_timestamp > "2018-09-01";


In [ ]:
#### JOINING DATE IN SQL to INCREASE TABLEAU PERFORMANCE after LOADING IT IN THERE

# CREATED a so called REVENUE_CUSTOMER Dashboard with all the relevant colums for all visualizations

In [ ]:
# SELECT `orders`.`order_status`, `orders`.`order_purchase_timestamp`, `orders`.`order_approved_at`, `orders`.`order_delivered_carrier_date`, `orders`.`order_delivered_customer_date`, `orders`.`order_estimated_delivery_date`, `order_customer_items_paid`.`order_id`,
#    `order_customer_items_paid`.`customer_city`,
#    `order_customer_items_paid`.`customer_id`,
#    `order_customer_items_paid`.`order_item_id`,
#    `order_customer_items_paid`.`price`,
#   `order_customer_items_paid`.`freight_value`,
#    `order_customer_items_paid`.`final_price`,
#    `customers`.`customer_zip_code_prefix`,
#    `customers`.`customer_state`,
#    `products`.`product_category_name`,
#    `order_payments`.`payment_installments`,
#    `order_payments`.`payment_type`,
#    `customers`.`customer_state_name`,
#    `order_items`.`seller_id`
# FROM olist.orders
# inner join `order_customer_items_paid`
# using (order_id)
# inner join customers
# on `customers`.`customer_id` = `orders`.`customer_id`
# inner join order_items
# on `order_items`.`order_id` = `order_customer_items_paid`.`order_id`
# inner join products
# on `products`.`product_id` = `order_items`.`product_id`
# inner join order_payments
# on `order_payments`.`order_id` = `order_customer_items_paid`.`order_id`;